## ITEM 1

In [ ]:
r = 10
R = [.193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]
x = [10, 10, 10, 10, 10]

In [2]:
# Test cell
10**(-2)

0.01

### Método de Newton

### Método de Broyden

### Comparação dos resultados

## ITEM 2